In [ ]:
from data_collection import (store_database_for_eys_gene,
                             parse_lovd,
                             LOVD_PATH,
                             set_lovd_dtypes)

In [ ]:
store_database_for_eys_gene("lovd", override=False)

In [ ]:
data = parse_lovd(LOVD_PATH + "/lovd_data.txt")
for i in data:
    print(i)
    display(data[i])

In [ ]:
set_lovd_dtypes(data)
for i in data:
    print(i)
    display(data[i].info())